

<h1><center>De la donnée aux bases de données - S2-23</center></h1>
<h1><center>TP noté - Ecrire dans une base de donnée</center></h1>




#### Nom étudiant 1 : Isabella Raevel
#### Nom étudiant 2 : Anais Storp 

# Sommaire 

- [  Introduction : description des données et quelle tâche y est associée](#chapter1)
    - [ Installation de SQLAlchemy et import des bibliothéques ](#section_1_1)
    - [ Le client SQL: création de fonction amélioratrices ](#section_1_2)
    - [ Exemples d'utilisations ](#section_1_3)
- [Creer une table et inserer des données programmatiquement](#chapter2)
  - [Étape 1: Création et de  la base et d'ue table acceuillant les données ](#section_2_1)
  - [Étape 2: Convertion des donnés et troubleshooting ](#section_2_2)
  - [Étape 3: Insertion des données dans la base](#section_2_3)
    - [Tests](#section_2_3_1)
  - [Étape 4: Amelioration des performances](#section_2_4)
  - [Étape 5: Méthode alternative](#section_2_5)


### <ins>Introduction : description des données et quelle tâche y est associée</ins> <a class="anchor" id="chapter1"></a>


Dans ce notebook, nous utiliserons SQLAlchemy, une bibliothèque Python populaire pour l'interaction avec les bases de données relationnelles. SQLAlchemy offre une approche flexible pour travailler avec les bases de données, permettant l'utilisation de langage SQL tout en bénéficiant des fonctionnalités de Python.

#### <ins>Installation de SQLAlchemy et import des bibliothéques </ins>  <a class="anchor" id="section_1_1"></a>


Premièrement nous installons SQLAlchemy et nous verifions si nous avons bien la version la plus récente. Ensuite nou simportant les bibliothèques nécessaires dans notre script Python: pandas est utilisée pour la manipulation et l'analyse de données; numpy nous sert à effectuer des calculs numériques sur des tableaux; mysql.connector founit des fonctionnalitßes du module sqlalchemy; create_engine et text sont importées à partir du module sqlalchemy. 


In [1]:
%pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import mysql.connector 
from sqlalchemy import create_engine
from sqlalchemy import text

In [3]:
%pip install --upgrade SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


#### <ins>Le client SQL: création de fonction amélioratrices</ins> <a class="anchor" id="section_1_2"></a>

On va "ameliorer" notre client en creant deux fonctions
- **execute_statement** qui va executer une statement SQL et ramener le result set si fetch_data=True (attention il faut que ce soit pertinent)
- **read_dataframe** : qui va executer une query "SELECT ..." et retourner un dataframe former avec le resultat

on s'assurera aussi explicitement que toutes les connections à MySQL sont bien fermées à la sortie de ces fonctions

In [4]:
from sqlalchemy import create_engine, text 

mysql_USERNAME = 'root'
mysql_PASSWORD = 'Paris2024*'
mysql_host = '127.0.0.1'
mysql_port = '3306'

def execute_statement(statement, 
                      db_name=None,
                      fetch_data=False,
                      echo=False
                     ):
    if db_name is None:
        my_db_connection_str = f'mysql+mysqlconnector://{mysql_USERNAME}:{mysql_PASSWORD}@{mysql_host}:{mysql_port}'
    else:
        my_db_connection_str = f'mysql+mysqlconnector://{mysql_USERNAME}:{mysql_PASSWORD}@{mysql_host}:{mysql_port}/{db_name}'

    db_connectable = create_engine(my_db_connection_str, echo=echo)
    with db_connectable.begin() as connection:
        statement = text(statement)
        cursor_result = connection.execute(statement)
        if fetch_data:
            return cursor_result  
        else:
            
            return None

   
    db_connectable.dispose()


In [5]:

def read_dataframe(query, db_name=None):
    if db_name is None:
         my_db_connection_str = 'mysql+mysqlconnector://{}:{}@{}'.format(mysql_USERNAME, mysql_PASSWORD, '127.0.0.1')
    else:
       my_db_connection_str = 'mysql+mysqlconnector://{}:{}@{}/{}'.format(mysql_USERNAME, mysql_PASSWORD, '127.0.0.1', db_name)

    
    db_connectable = create_engine(my_db_connection_str)
    with db_connectable.begin() as connection:
        query = text(query)
        df = pd.read_sql_query(query,  connection)

    # Fermer toutes les connexions après l'exécution de la requête
        db_connectable.dispose()
        
    return df


#### <ins>Exemples d'utilisations</ins>  <a class="anchor" id="section_1_3"></a>

Les exemples d'utilisation des fonctions read_dataframe et execute_statement illustrent comment ces fonctions peuvent être utilisées pour interagir avec une base de données MySQL à l'aide de SQLAlchemy.

In [6]:
# lire un dataframe en specifiant dans la connection le nom de la database
query = 'SELECT * FROM actor'

df = read_dataframe(query, db_name='sakila')
df.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [7]:
# lire un dataframe sans specifiant dans la connection le nom de la database
query = 'SELECT * FROM sakila.actor'

df = read_dataframe(query, db_name=None)
df.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


<div class="alert alert-block alert-info">
<b>Utilisation de read_dataframe:</b> 
La fonction read_dataframe est conçue pour exécuter une requête SQL SELECT sur une base de données MySQL et renvoyer les résultats sous forme de DataFrame pandas. Dans l'exemple, nous utilisons cette fonction pour sélectionner toutes les données de la table "planet" et les stocker dans un DataFrame appelé df_from_db. Cela permet de manipuler facilement et d'analyser les données récupérées à partir de la base de données, car les DataFrames pandas offrent de nombreuses fonctionnalités pour travailler avec les données tabulaires.
</div>

In [8]:
# recuperer directement les données d'un select en specifiant dans la database dans la connection
statement = 'select * from actor'
execute_statement(statement, fetch_data=True, db_name='sakila')

In [9]:
# recuperer les données d'un select en specifiant dans la database dansle statement SQL
statement = 'select * from sakila.actor'
execute_statement(statement, fetch_data=True, db_name=None)

In [10]:
# liste des tables dans une database
statement = 'SHOW TABLES'
execute_statement(statement, fetch_data=True, db_name='sakila')

<div class="alert alert-block alert-info">
<b>Utilisation de execute_statement:</b> 
La fonction execute_statement est utilisée pour exécuter une requête SQL quelconque sur une base de données MySQL. Dans l'exemple, nous utilisons cette fonction pour exécuter une instruction SQL DELETE afin de supprimer toutes les données de la table "planet". Cela peut être utile dans divers scénarios où nous devons effectuer des opérations d'écriture sur la base de données, telles que l'insertion, la mise à jour ou la suppression de données.
</div>

In [11]:
# creation d'une database test 
statement = 'DROP DATABASE IF EXISTS test'
execute_statement(statement, fetch_data=False, db_name=None)

statement = 'CREATE DATABASE test'
execute_statement(statement, fetch_data=False, db_name=None)

In [12]:
#creer une table dans la database test

statement = 'SHOW TABLES'
print(execute_statement(statement, fetch_data=True, db_name='test'))


statement = 'DROP TABLE IF EXISTS test.countries;'
execute_statement(statement, fetch_data=False, db_name='test')

statement = 'CREATE TABLE countries (country_id CHAR(2) NOT NULL, country_name VARCHAR(40), region_id INT (11) UNSIGNED NOT NULL,PRIMARY KEY (country_id));'

execute_statement(statement, fetch_data=False, db_name='test')

print(execute_statement('SHOW TABLES', fetch_data=True, db_name='test'))

In [13]:
# inserer une ligne
statement = "INSERT INTO countries VALUES ('IT','Italy',1);"
execute_statement(statement, fetch_data=False, db_name='test')

In [14]:
# inserer des lignes
statement = "INSERT INTO countries VALUES  ('JP', 'Japan', 3), ('US', 'United States of America', 2);"
execute_statement(statement, fetch_data=False, db_name='test')

###  <ins>Creer une table et inserer des données programmatiquement</ins>  <a class="anchor" id="chapter2"></a>

---

le but est de crée une table programatiquement avec les données du catalog des exo planets de trois manieres

1. en inserant les données ligne par ligne
2. en inserant les données en "batch"
3. LOAD DATA pour chargé les données d'un fichier csv

pour nous faciliter la vie nous allons "tricher" et commencer par faire lire le csv par pandas

In [15]:
import pandas as pd

df = pd.read_csv("exoplanets.csv") #Nous trichant en tßelechargeant le dossier csv localement pour le lire directement avec pandas

cols = ['name', 'mass', 'radius', 'orbital_period', 'eccentricity', 'discovered', 'detection_type',
        'star_name', 'star_distance', 'star_mass']
df = df[cols] #Nous créons le dataframe avec les colonnes choisies

print(df.shape) #Nous obtenons un dataframes a 5586 lignes et 10 colonnes


(5586, 10)


<div class="alert alert-block alert-success">
<b>Simplification: </b> Nous allons considerer que toutes les colonnes numeriques sont de type FLOAT et 
que toutes les colonnes non numeriques sont de types VARCHAR(225)
Pour faire la difference nous utiliserons la fonction DataFrames.select_dtypes comme ci dessous pour recuperer la liste des colonnes numeriques et la listes des colonnes non numeriques
</div>


In [16]:
cols_num = df.select_dtypes(include=np.number).columns.tolist()
cols_notnum = df.select_dtypes(exclude=np.number).columns.tolist()

####  <ins>Étape 1: Création et de  la base et d'ue table acceuillant les données </ins>  <a  id="section_2_1"></a>
___

Creer une DATABASE de test et dans cette base creer une table planet ayant les memes colonnes que df.

Pour cela il faut commencer par generer de maniere programatique le statement SQL en se servant des variables cols_num pour determiner les types puis l'executer.

In [17]:
# creation d'une database test 
statement = 'DROP DATABASE IF EXISTS planetes' #On "drop" tout database déjà existant sous le même nom pour éviter les problèmes/confusions
execute_statement(statement, fetch_data=False, db_name=None)

statement = 'CREATE DATABASE planetes' #On crée notre base de données et on l'a nomme "planetes"
execute_statement(statement, fetch_data=False, db_name=None)



col_types = {
    'name': 'VARCHAR(255)',  # Nous avons définis une longeure maximale arbitrairement 
    'mass': 'FLOAT',
    'radius': 'FLOAT',
    'orbital_period': 'FLOAT',
    'eccentricity': 'FLOAT',
    'discovered': 'FLOAT',  # Ici on a utilisé float et non Year puisque les années sont données avec une décimale dans le fichier csv
    'detection_type': 'VARCHAR(255)',  #Nous avons choisi une longeure maximale très importante pour éviter les problèmes après
    'star_name': 'VARCHAR(255)',
    'star_distance': 'FLOAT',
    'star_mass': 'FLOAT'
}
#creer une table dans la database test
statement = 'SHOW TABLES'
print(execute_statement(statement, fetch_data=True, db_name='planetes'))

statement = 'DROP TABLE IF EXISTS planetes.planet;' #On verifie que la table planet n'existe pas 
execute_statement(statement, fetch_data=False, db_name='test')

create_table_sql = 'CREATE TABLE planet ('
for col_name, col_type in col_types.items():
    create_table_sql += f'{col_name} {col_type}, '
create_table_sql = create_table_sql[:-2] + ')'  # on supprime la dernière virgule et l'espace

# on exécute le code SQL pour créer la table dans la base de données planetes
execute_statement(create_table_sql, fetch_data=False, db_name='planetes')

# on vérifie bien que la table a bien été créée
print(execute_statement('SHOW TABLES', fetch_data=True, db_name='planetes'))



####  <ins>Étape 2: Convertion des donnés et troubleshooting </ins>  <a class="anchor" id="section_2_2"></a>
Ensuite pour inserer ligne par ligne on creer une fonction gen_st_row(df, row_number, cols_num) qui prend argument

___
- un dataframe df
- le numero de la ligne considerer
- la liste des colonnes numeriques (les colonnes non numeriques sont reputées etre des colonnes de type chaine de caracteres)
et qui renvoie le statement insert correspondant.

Il faudra prendre garde de 
1. transformer les NaN en NULL (on poura se servir de la fonction pd.isna pour les reconnaitre)
2. mettre les valeurs de type  VARCHAR entre quote

___
Par exemple, le statement SQL correspondant la premier ligne de df sera

INSERT INTO planet VALUES ('11 Com b', 16.1284, NULL, 326.03, 0.231, 2007.0, 'Radial Velocity', '11 Com', 110.6, 2.7)



In [18]:
def gen_st_row(df, row_number, cols_num):
    # on sélectionne la ligne correspondante dans le DataFrame
    row = df.iloc[row_number]

    # on initialise le statement d'insertion
    insert_statement = "INSERT INTO planet VALUES ("

    # on parcourt chaque colonne du DataFrame
    for col in df.columns:
        # on vérifie si la colonne est numérique ou non
        if col in cols_num:
            # Si la valeur est NaN, la remplacer par NULL, sinon on laisse la valeur
            value = str(row[col]) if not pd.isna(row[col]) else "NULL"
        else:
            # Si la valeur est NaN, la remplacer par NULL
            # Si la valeur contient des single quotes, les remplacer par double single quotes
            value = row[col] if not pd.isna(row[col]) else "NULL"
            if isinstance(value, str):
                value = value.replace("'", "''")
            value = f"'{value}'"
        
        # on ajoute la valeur au statement d'insertion
        insert_statement += f"{value}, "

    # on supprime la virgule finale et ajouter la parenthèse qui ferme
    insert_statement = insert_statement[:-2] + ")"
    
    return insert_statement


####  <ins>Étape 3: Insertion des données dans la base</ins>  <a class="anchor" id="section_2_3"></a>
Ensuite il suffit d'iterer sur le numero de ligne de df, pour "remplir" la table ligne par ligne. 
On utilisera %%time pour estimer le temps d'execution.

In [19]:
from IPython.display import display #pour faire jolie
from tqdm import tqdm #tqdm est une bibliothèque Python qui fournit une barre de progression élégante et intuitive pour les boucles itératives
import time

# on définit le nombre total de lignes dans le DataFrame
total_rows = len(df)

# on utilise %%time pour mesurer le temps d'exécution
start_time = time.time()

# on insère sur chaque numéro de ligne dans le DataFrame
for row_number in tqdm(range(total_rows)):
    # on genere le statement d'insertion pour la ligne actuelle
    insert_statement = gen_st_row(df, row_number, cols_num)
    
    # on execute le statement d'insertion dans la base de données
    execute_statement(insert_statement, fetch_data=False, db_name='planetes')

# Afficher le temps d'exécution
end_time = time.time()
execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time} secondes")


  0%|          | 6/5586 [00:00<08:41, 10.70it/s]

100%|██████████| 5586/5586 [05:02<00:00, 18.47it/s]

Temps d'exécution : 302.50287103652954 secondes


Au cas où (en cas de probleme) pour redemarer MySQL
- sur ubuntu et debian: sudo service mysql restart
- sudo systemctl restart mysqld

Sur Windows:
- Ouvrez le "Gestionnaire des services" en appuyant sur Win + R, tapez services.msc, puis appuyez sur Entrée.
- Recherchez le service MySQL, cliquez avec le bouton droit de la souris dessus, puis sélectionnez "Redémarrer".


**Verifier que les données sont correctement inserées**

In [20]:
# on definit la requête de sélection pour afficher quelques lignes de la table planet
select_query = 'SELECT * FROM planet LIMIT 5'  # Sélectionner les 5 premières lignes

# on execute la requête de sélection et stocker les résultats dans un DataFrame
result_df = read_dataframe(select_query, db_name='planetes')

# Afficher le DataFrame contenant les données insérées
print("Données insérées dans la table planet :\n") #\n c'est pour reveneir à la ligne
display(result_df)


Données insérées dans la table planet :



,name,mass,radius,orbital_period,eccentricity,discovered,detection_type,star_name,star_distance,star_mass
0,11 Com b,16.1284,None,326.03,0.231,2007.0,Radial Velocity,11 Com,110.6,2.7000
1,11 Oph b,21.0000,None,730000.00,NaN,2007.0,Imaging,11 Oph,145.0,0.0162
2,11 UMi b,11.0873,None,516.22,0.080,2009.0,Radial Velocity,11 UMi,119.5,1.8000
3,14 And b,4.6840,None,185.84,0.000,2008.0,Radial Velocity,14 And,76.4,2.2000
4,14 Her b,NaN,None,1767.56,0.372,2002.0,Radial Velocity,14 Her,18.1,0.9000


#### <ins> Tests </ins>  <a class="anchor" id="section_2_3_1"></a>
Pour tester on va lire un le contenue de la table planet dans un nouveau dataframe, df2

In [21]:
df2 = read_dataframe("SELECT * FROM planet", db_name='planetes')
df2 = df2.drop_duplicates() # on supprime tous les doublons dans la df2 si il y en a 
df2.replace('NULL', np.nan, inplace=True) # on change tout les NUll en Nan 
df2

,name,mass,radius,orbital_period,eccentricity,discovered,detection_type,star_name,star_distance,star_mass
0,11 Com b,16.1284,NaN,326.030000,0.231,2007.0,Radial Velocity,11 Com,110.6,2.7000
1,11 Oph b,21.0000,NaN,730000.000000,NaN,2007.0,Imaging,11 Oph,145.0,0.0162
2,11 UMi b,11.0873,NaN,516.220000,0.080,2009.0,Radial Velocity,11 UMi,119.5,1.8000
3,14 And b,4.6840,NaN,185.840000,0.000,2008.0,Radial Velocity,14 And,76.4,2.2000
4,14 Her b,NaN,NaN,1767.560000,0.372,2002.0,Radial Velocity,14 Her,18.1,0.9000
...,...,...,...,...,...,...,...,...,...,...
5581,ZTFJ0220+21 b,14.0000,0.54,0.037000,NaN,2021.0,Primary Transit,ZTFJ0220+21,350.0,0.8300
5582,ZTFJ0407-00 b,19.0000,0.44,0.024600,NaN,2021.0,Primary Transit,ZTFJ0407-00,810.0,0.7900
5583,ZTF-J1622+47 b,61.0000,0.98,0.069789,NaN,2022.0,Primary Transit,ZTF-J1622+47,NaN,0.4700
5584,ZTFJ1637+49 b,23.0000,0.68,0.043000,NaN,2021.0,Primary Transit,ZTFJ1637+49,207.0,0.9000


1. on verifie (programatiquement) que df et df2 ont meme tailles, meme noms de colonnes et memes contenus apres avoir arrondi les numeriques

In [22]:
#compare deux colonnes de meme tailles
#s1 et s2 sont des Series
def compare_columns(s1, s2):
    if len(s1)!=len(s2):
        print("size mismatch")
        return False
    if s1.dtypes != s2.dtypes:
        print("type mismatch")
        return False

    for i in range(len(s1)):
        if pd.isna(s1[i]) and pd.isna(s2[i]):
            continue
        if pd.isna(s1[i]) or pd.isna(s2[i]):
            print("issues with element", i)
            return False
        #en cas de valeurs numeriques on arrondi
        if pd.api.types.is_number(s1[i]) and round(s1[i],0) != round(s2[i], 0):
            print("issues with element", i)
            return False
        if not (pd.api.types.is_number(s1[i])) and s1[i] != s2[i]:
            print("issues with element", i)
            return False 

    return True



In [23]:
# Test 1

df = pd.read_csv("exoplanets.csv")

cols = ['name', 'mass', 'radius', 'orbital_period', 'eccentricity', 'discovered', 'detection_type',
        'star_name', 'star_distance', 'star_mass']
df = df[cols]

# Read data from the database, convert 'NULL' to NaN, and drop duplicates
df2 = read_dataframe("SELECT * FROM planet", db_name='planetes')
df2 = df2.replace('NULL', np.nan)
df2 = df2.drop_duplicates()

# Verify each column
ok = True
for col in df.columns:
    if not compare_columns(df[col], df2[col]):
        print("problem with column", col)
        ok=False
if ok:
    print("all ok (after rounding)")


all ok (after rounding)


2. Qu'est ce qui se passe si on n'omet d'arrondir ? Comment pourrait on resoudre ce probleme ? 
<div class="alert alert-block alert-warning">
<b>Arrondir:</b> 
Si nous omettons d'arrondir les valeurs numériques lors de la comparaison entre les colonnes de df et df2, cela peut entraîner des différences dans les résultats de la comparaison, car les valeurs numériques peuvent différer légèrement en raison de précisions différentes lors de leur stockage dans la base de données.

Pour résoudre ce problème, nous pouvons arrondir les valeurs numériques dans les deux DataFrames avant de les comparer. Cela garantit que les valeurs sont comparées de manière équitable en ignorant les différences mineures dues à la précision de stockage.
</div>

In [24]:
# Test 1

df = pd.read_csv("exoplanets.csv")

cols = ['name', 'mass', 'radius', 'orbital_period', 'eccentricity', 'discovered', 'detection_type',
        'star_name', 'star_distance', 'star_mass']
df = df[cols]

# on lit la data de la database, on convertit 'NULL' en NaN, et on drop les duplicats
df2 = read_dataframe("SELECT * FROM planet", db_name='planetes')
df2 = df2.replace('NULL', np.nan)
df2 = df2.drop_duplicates()

# on  arrondit les numériquesdans les deux dataframes 
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
df[numeric_cols] = df[numeric_cols].round(0)
df2[numeric_cols] = df2[numeric_cols].round(0)

# on verfiei chaque colonne
ok = True
for col in df.columns:
    if not compare_columns(df[col], df2[col]):
        print("problem with column", col)
        ok=False
if ok:
    print("all ok (after rounding)")


all ok (after rounding)


####  <ins>Étape 4: Amelioration des performances</ins>  <a class="anchor" id="section_2_4"></a>
On pourrais rendre cela plus efficace en reutilisant la connection à MySQL pour chaque statement. Mais on devrait pourvoir faire encore 
mieux en generant un statement unique pour inserer toutes les données d'un coup.
Faites le, executez les tests et comparer les temps d'execution et tester la correction du resultat


In [31]:
def gen_insert_statement(df, cols_num):
    # Initialize the insertion statement
    insert_statement = "INSERT INTO planet VALUES "

    # Iterate over each row of the DataFrame
    for index, row in df.iterrows():
        insert_statement += "("

        # Iterate over each column of the DataFrame
        for col in df.columns:
            # Check if the column is numeric or not
            if col in cols_num:
                # Replace NaN with NULL for numeric columns
                value = str(row[col]) if not pd.isna(row[col]) else "NULL"
            else:
                # Replace NaN with NULL and handle single quotes
                value = row[col] if not pd.isna(row[col]) else "NULL"
                if isinstance(value, str):
                    value = value.replace("'", "''")
                value = f"'{value}'"

            # Add the value to the insertion statement for this row
            insert_statement += f"{value}, "

        # Remove the final comma and add the closing parenthesis for this row
        insert_statement = insert_statement[:-2] + "), "

    # Remove the final comma and add a semicolon at the end of the insertion statement
    insert_statement = insert_statement[:-2] + ";"

    return insert_statement


In [46]:
# Measure the execution time of the optimized insertion method
start_time = time.time()

# Generate the single insertion statement and execute it
insert_statement = gen_insert_statement(df, cols_num)
execute_statement(insert_statement, fetch_data=False, db_name='planetes')
df.drop_duplicates()

# Calculate the execution time
end_time = time.time()
execution_time_optimized = end_time - start_time

# Compare the execution times
print(f"Execution time (optimized): {execution_time_optimized} seconds")
print(f"Execution time (previous): {execution_time} seconds")

# Read data from the database and ensure consistency with df
df_from_db = read_dataframe("SELECT * FROM planet", db_name='planetes')
#df_from_db.replace('NULL', np.nan, inplace=True)
df_from_db = df_from_db.drop_duplicates()

if df.equals(df_from_db):
    print("Inserted data matches the original DataFrame.")
else:
    print("Inserted data does not match the original DataFrame.")

KeyboardInterrupt: 

In [45]:
print(df_from_db.shape)
print(df.shape)

(11124, 10)
(11172, 10)


### <ins>Introduction : description des données et quelle tâche y est associée</ins> <a class="anchor" id="chapter1"></a>


On va "ameliorer" notre client en creant deux fonctions
- execute_statement qui va executer une statement SQL et ramener le result set si fetch_data=True (attention il faut que ce soit pertinent)
- read_dataframe : qui va executer une query "SELECT ..." et retourner un dataframe former avec le resultat

on s'assurera aussi explicitement que toutes les connections à MySQL sont bien fermées à la sortie de ces fonctions

####  <ins>Étape 5: Méthode alternative</ins>  <a class="anchor" id="section_2_5"></a>

____

Enfin apres avoir exportant df vers un fichier csv de maniere appropriée (en specifiant les caracteristiques du dialectes CSV et 
en evitant de creer une colonne pour l'indexe) utiliser LOAD DATA INFILE pour creer la table et comparer les temps d'execution.
Tester la correction des resultats 

In [28]:
import pandas as pd
from sqlalchemy import create_engine, text as sql_text
import csv

username = 'root'
password = 'Paris2024*'
host = 'localhost'
port = '3306'
database = 'planetes'

# Create a connection to the MySQL database
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')

# Load data from the 'planet' table in the database into a DataFrame
df = read_dataframe("SELECT * FROM planet", db_name='planetes')

# Export the DataFrame to a CSV file
csv_filename = "exoplanets_export.csv"
df.to_csv(csv_filename, index=False, quoting=csv.QUOTE_NONNUMERIC)

# Insert the data from the DataFrame into the 'planet' table in the database
df.to_sql('planet', con=engine, if_exists='replace', index=False)

# Verify that the data was inserted correctly by reading it back from the database
query = "SELECT * FROM planet"
df_from_db = pd.read_sql(con=engine.connect(), sql=sql_text(query))
if df.equals(df_from_db):
    print("Inserted data matches the original DataFrame.")
else:
    print("Inserted data does not match the original DataFrame.")


Inserted data matches the original DataFrame.
